# 종합 통계 실습 — Customer Personality Analysis

**비즈니스 시나리오**

> 여러분은 식품·와인 유통 기업 **FreshCart**의 데이터 분석팀 인턴입니다.
> 마케팅팀이 6차례 캠페인을 진행했지만 수락률이 낮아 경영진이 우려하고 있습니다.
> 고객 데이터를 분석하여 **다음 캠페인의 집중 공략 전략**을 수립해 주세요.

---

| Part | 주제 | 핵심 질문 |
|------|------|-----------|
| **Part 0** | 데이터 전처리 | (코드 제공) |
| **Part 1** | 탐색적 데이터 분석 | 고객은 누구인가? |
| **Part 2** | 확률분포 모델링 | 고객 행동은 어떤 패턴을 따르는가? |
| **Part 3** | 추정과 가설검정 | 고소득 고객과 저소득 고객의 소비는 다른가? |
| **Part 4** | 고급 검정 | 교육, 결혼, 캠페인 반응의 관계는? |
| **Part 5** | 상관분석 / 비율 비교 | 변수 간 관계를 파악하고, 캠페인 효과를 검증할 수 있는가? |
| **Challenge** | 종합 분석 보고서 | 다음 캠페인 타겟은 누구인가? |

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import trim_mean
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.proportion import (
    proportions_ztest, proportion_confint, proportion_effectsize
)
from statsmodels.stats.power import NormalIndPower, TTestIndPower
import pingouin as pg
import warnings
import platform

warnings.filterwarnings('ignore')


# --- 한글 폰트 설정 (OS별 자동 분기) ---
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':
    plt.rcParams['font.family'] = 'AppleGothic'
else:
    plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 부호 깨짐 방지


# --- Tailwind CSS 기반 색상 팔레트 ---
COLORS = {
    'blue': '#3B82F6', 'indigo': '#6366F1', 'violet': '#8B5CF6',
    'teal': '#14B8A6', 'emerald': '#10B981', 'amber': '#F59E0B',
    'rose': '#F43F5E', 'red': '#EF4444', 'slate': '#64748B',
    'sky': '#0EA5E9', 'purple': '#A855F7', 'lime': '#84CC16',
}

---
## Part 0: 데이터 전처리 (코드 제공)

아래 코드를 실행하여 분석에 사용할 데이터프레임 `df`를 준비합니다.
모든 파생변수가 생성되고 이상치가 제거된 상태에서 Part 1부터 분석을 시작합니다.

### 0-1. 데이터 로드 및 결측치 처리

In [ ]:
df = pd.read_csv('data/marketing_campaign.csv', sep='\t')
print(f"원본 데이터: {df.shape[0]}행 × {df.shape[1]}열")
print(f"Income 결측치: {df['Income'].isnull().sum()}개")

df['Income'] = df['Income'].fillna(df['Income'].median())
print(f"→ Income 중앙값({df['Income'].median():,.0f})으로 대치 완료")

### 0-2. 파생변수 생성

In [ ]:
# ── 원본 컬럼 그룹 정의 ──
spending_cols = ['MntWines', 'MntFruits', 'MntMeatProducts',
                 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
purchase_cols = ['NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases']
campaign_cols = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3',
                 'AcceptedCmp4', 'AcceptedCmp5', 'Response']

# ── 파생변수 생성 ──
df['Age'] = 2024 - df['Year_Birth']                        # 나이 (출생연도 기준)
df['Total_Spending'] = df[spending_cols].sum(axis=1)        # 6개 카테고리 총 지출액
df['Total_Purchases'] = df[purchase_cols].sum(axis=1)       # 웹+카탈로그+매장 총 구매 횟수
df['Total_Accepted'] = df[campaign_cols].sum(axis=1)        # 수락한 캠페인 수 (0~6)
df['Has_Children'] = (df['Kidhome'] + df['Teenhome']) > 0   # 자녀 유무 (True/False)
df['Income_Group'] = pd.qcut(df['Income'], 3,
                             labels=['Low', 'Mid', 'High']) # 소득 3분위 그룹

bins = [0, 39, 49, 59, 200]
labels_age = ['30대 이하', '40대', '50대', '60대 이상']
df['Age_Group'] = pd.cut(df['Age'], bins=bins,
                         labels=labels_age)                 # 연령대 구간 그룹

edu_map = {'Basic': 'Undergraduate', '2n Cycle': 'Undergraduate',
           'Graduation': 'Graduate', 'Master': 'Postgraduate', 'PhD': 'Postgraduate'}
df['Education_Group'] = df['Education'].map(edu_map)        # 학력 3단계 (학부/졸업/대학원)

marital_map = {'Married': 'Together', 'Together': 'Together',
               'Single': 'Single', 'Divorced': 'Single', 'Widow': 'Single',
               'Alone': 'Single', 'Absurd': 'Single', 'YOLO': 'Single'}
df['Marital_Group'] = df['Marital_Status'].map(marital_map) # 결혼 상태 2분류 (동거/독신)

df['Campaign_Response'] = df['Total_Accepted'] > 0          # 캠페인 1회 이상 수락 여부

df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%d-%m-%Y')
reference_date = df['Dt_Customer'].max()
df['Customer_Days'] = (reference_date - df['Dt_Customer']).dt.days  # 가입 이후 경과 일수


### 0-3. 이상치 제거

In [ ]:
n_before = len(df)
df = df[(df['Age'] <= 100) & (df['Income'] <= 200000)].copy()
n_after = len(df)
print(f"이상치 제거: {n_before}행 → {n_after}행 ({n_before - n_after}행 제거)")
print(f"\n최종 데이터 요약:")
print(f"  Age 범위: {df['Age'].min()} ~ {df['Age'].max()}")
print(f"  Income 범위: {df['Income'].min():,.0f} ~ {df['Income'].max():,.0f}")
print(f"\n데이터 준비가 완료되었습니다. Part 1부터 분석을 시작하세요!")

---
## Part 1: 탐색적 데이터 분석 — "고객은 누구인가?"

기술통계량과 시각화를 통해 FreshCart 고객의 특성을 파악합니다.

### 문제 1-1. 중심경향과 산포도

`Income`, `Total_Spending`, `Age` 세 변수에 대해 다음을 계산하세요.

| 중심경향 | 산포도 |
|----------|--------|
| 평균(mean) | 분산(variance) |
| 중앙값(median) | 표준편차(std) |
| 최빈값(mode) | 사분위범위(IQR) |
| 5% 절사평균(trimmed mean) | 변동계수(CV) |

In [ ]:
from scipy import stats
from scipy.stats import trim_mean

target_vars = ['Income', 'Total_Spending', 'Age']


result = df[target_vars].agg(lambda x: pd.Series({
    '평균':     x.mean(),
    '중앙값':   x.median(),
    '최빈값':   x.mode().iloc[0],
    '절사평균': trim_mean(x.dropna(), 0.05),
    '분산':     x.var(ddof=1),
    '표준편차': x.std(ddof=1),
    'IQR':      x.quantile(0.75) - x.quantile(0.25),
    'CV(%)':    x.std(ddof=1) / x.mean() * 100,
    '왜도':     stats.skew(x.dropna()),
}))

print("[기초 통계량]")
print(result.to_string())

In [ ]:
from scipy import stats
from scipy.stats import trim_mean

target_vars = ['Income', 'Total_Spending', 'Age']
group_col = 'Education'

result = (
    df.groupby(group_col)[target_vars]
    .agg([
        ('평균',     lambda x: x.mean()),
        ('중앙값',   lambda x: x.median()),
        ('최빈값',   lambda x: x.mode().iloc[0]),
        ('절사평균', lambda x: trim_mean(x.dropna(), 0.05)),
        ('분산',     lambda x: x.var(ddof=1)),
        ('표준편차', lambda x: x.std(ddof=1)),
        ('IQR',      lambda x: x.quantile(0.75) - x.quantile(0.25)),
        ('CV(%)',    lambda x: x.std(ddof=1) / x.mean() * 100),
        ('왜도',     lambda x: stats.skew(x.dropna())),
    ])
)

# 멀티인덱스 컬럼: (변수, 통계량)
# result['Income']          → Income의 모든 통계량
# result['Income']['평균']  → Income 평균만

print("[문제 1-1] 중심경향과 산포도 (그룹별)")
print("=" * 60)

for var in target_vars:
    print(f"\n▶ {var}")
    print(result[var].to_string())

In [ ]:
target_vars = ['Income', 'Total_Spending', 'Age']

print("[문제 1-1] 중심경향과 산포도")
print("=" * 60)

for var in target_vars:
    data = df[var]
    mean_val = data.mean()
    median_val = data.median()
    mode_val = data.mode().iloc[0]
    trimmed_val = trim_mean(data, 0.05)

    var_val = data.var(ddof=1)
    std_val = data.std(ddof=1)
    q1, q3 = data.quantile(0.25), data.quantile(0.75)
    iqr_val = q3 - q1
    cv_val = std_val / mean_val * 100

    print(f"\n▶ {var}")
    print(f"  [중심경향]")
    print(f"    평균:       {mean_val:>12,.2f}")
    print(f"    중앙값:     {median_val:>12,.2f}")
    print(f"    최빈값:     {mode_val:>12,.2f}")
    print(f"    절사평균:   {trimmed_val:>12,.2f}")
    print(f"  [산포도]")
    print(f"    분산:       {var_val:>12,.2f}")
    print(f"    표준편차:   {std_val:>12,.2f}")
    print(f"    IQR:        {iqr_val:>12,.2f}")
    print(f"    CV:         {cv_val:>11.1f}%")

    skew = stats.skew(data)
    if abs(skew) < 0.2:
        skew_comment = "거의 대칭에 가까운 분포입니다."
    elif skew > 0:
        skew_comment = f"약한 오른쪽(양의) 비대칭입니다 (왜도={skew:.3f})."
    else:
        skew_comment = f"약한 왼쪽(음의) 비대칭입니다 (왜도={skew:.3f})."
    if abs(skew) >= 0.5:
        skew_comment = skew_comment.replace("약한 ", "뚜렷한 ")
    diff_pct = abs(mean_val - median_val) / mean_val * 100
    print(f"  → 왜도={skew:.3f}, 평균-중앙값 차이={mean_val - median_val:,.1f} ({diff_pct:.1f}%)")
    print(f"    {skew_comment}")

### 문제 1-2. 분포 시각화

- **(a)** Income 히스토그램 + KDE
- **(b)** Total_Spending 박스플롯 + 이상치 표시
- **(c)** Income_Group별 Total_Spending 바이올린 플롯

In [ ]:
# (a) Income 히스토그램 + KDE — sns.histplot으로 히스토그램과 KDE를 동시에 그립니다
fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(df['Income'], bins=40, kde=True, stat='density',
             color=COLORS['blue'], edgecolor='white', alpha=0.6, ax=ax)
ax.axvline(df['Income'].mean(), color=COLORS['amber'], ls='--', lw=1.5,
           label=f"평균: {df['Income'].mean():,.0f}")
ax.axvline(df['Income'].median(), color=COLORS['emerald'], ls=':', lw=1.5,
           label=f"중앙값: {df['Income'].median():,.0f}")
ax.set(xlabel='Income', ylabel='밀도',
       title='Income 분포 (히스토그램 + KDE)')
ax.legend(fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
# (b) Total_Spending 박스플롯 + 이상치 탐지 — sns.boxplot으로 간결하게 표현합니다
fig, ax = plt.subplots(figsize=(10, 4))
sns.boxplot(x=df['Total_Spending'], orient='h', color=COLORS['emerald'],
            width=0.5, flierprops={'marker': 'o', 'alpha': 0.4}, ax=ax)

# IQR 기반 이상치 기준선 계산
q1 = df['Total_Spending'].quantile(0.25)
q3 = df['Total_Spending'].quantile(0.75)
iqr = q3 - q1
upper_fence = q3 + 1.5 * iqr
outliers = df[df['Total_Spending'] > upper_fence]

# Q1, Q3, 상한 위치 표시
for val, label, c in [(q1, f'Q1={q1:.0f}', COLORS['blue']),
                       (q3, f'Q3={q3:.0f}', COLORS['blue']),
                       (upper_fence, f'상한={upper_fence:.0f}', COLORS['rose'])]:
    ax.annotate(label, xy=(val, 0), xytext=(val, 0.3),
                fontsize=9, ha='center', color=c)
ax.set(xlabel='Total_Spending', title='Total_Spending 박스플롯')
plt.tight_layout()
plt.show()

print(f"IQR 기준 이상치: {len(outliers)}개 (상한 {upper_fence:.0f} 초과)")

In [ ]:
# (c) Income_Group별 Total_Spending 바이올린 플롯
#     sns.violinplot은 범주형 변수 + 연속형 변수를 자동으로 분리해 줍니다
fig, ax = plt.subplots(figsize=(10, 6))
sns.violinplot(data=df, x='Income_Group', y='Total_Spending',
               order=['Low', 'Mid', 'High'], inner='quartile',
               palette=[COLORS['sky'], COLORS['amber'], COLORS['emerald']], ax=ax)
ax.set(xlabel='Income Group', ylabel='Total Spending',
       title='소득 그룹별 소비 분포 (바이올린 플롯)')
plt.tight_layout()
plt.show()

### 문제 1-3. 왜도와 첨도

6개 소비 카테고리(MntWines ~ MntGoldProds)의 왜도(Skewness)와 첨도(Kurtosis)를 계산하고,
2×3 히스토그램 그리드를 그려 분포 형태를 비교하세요.

In [ ]:
print("[문제 1-3] 소비 카테고리별 왜도/첨도")
print("=" * 60)

spending_names = ['Wines', 'Fruits', 'Meat', 'Fish', 'Sweets', 'Gold']
skew_kurt_data = []

for col, name in zip(spending_cols, spending_names):
    s = stats.skew(df[col])
    k = stats.kurtosis(df[col])
    skew_kurt_data.append({'카테고리': name, '왜도': s, '첨도': k})
    shape = "양의 비대칭" if s > 0.5 else ("음의 비대칭" if s < -0.5 else "대칭에 가까움")
    tail = "뾰족한(Leptokurtic)" if k > 0 else "평평한(Platykurtic)"
    print(f"  {name:>7}: 왜도={s:+.3f} ({shape}), 첨도={k:+.3f} ({tail})")

print(f"\n→ 6개 카테고리 모두 왜도 > 1.0으로 강한 양의 비대칭(오른쪽 꼬리)입니다.")
print("  대다수 고객의 소비가 낮은 구간에 몰려 있고, 소수의 고소비 고객이 긴 꼬리를 형성합니다.")
print("  첨도 역시 모두 양수로 정규분포보다 꼬리가 두터운(Leptokurtic) 형태입니다.")

In [ ]:
# 2×3 히스토그램 그리드 — 각 소비 카테고리별 분포 형태를 한눈에 비교합니다
hist_colors = [COLORS['blue'], COLORS['violet'], COLORS['teal'],
               COLORS['emerald'], COLORS['amber'], COLORS['rose']]

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
for ax, col, name, color in zip(axes.ravel(), spending_cols, spending_names, hist_colors):
    s, k = stats.skew(df[col]), stats.kurtosis(df[col])
    sns.histplot(df[col], bins=30, color=color, edgecolor='white', alpha=0.6, ax=ax)
    ax.axvline(df[col].mean(), color=COLORS['red'], ls='--', lw=1.5)
    ax.set_title(f'{name}\n(왜도={s:.2f}, 첨도={k:.2f})', fontsize=11, fontweight='bold')
    ax.set(xlabel='지출액', ylabel='빈도')

plt.suptitle('소비 카테고리별 분포', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### 문제 1-4. 비즈니스 인사이트 정리

Part 1의 분석 결과를 바탕으로 FreshCart 고객의 특성을 요약하고,
후속 분석에서 검증할 가설을 제시하세요.

In [ ]:
print("[문제 1-4] 비즈니스 인사이트")
print("=" * 60)

inc_skew = stats.skew(df['Income'])
sp_skew = stats.skew(df['Total_Spending'])
sp_cv = df['Total_Spending'].std() / df['Total_Spending'].mean() * 100

print("\n1. 고객 특성 요약:")
print(f"   - 평균 연령 {df['Age'].mean():.1f}세의 중장년 고객이 주류입니다.")
print(f"   - 중위 소득 {df['Income'].median():,.0f}원으로, 소득 분포는 왜도={inc_skew:.2f}로 거의 대칭(종 모양)입니다.")
print(f"   - 총 소비는 평균 {df['Total_Spending'].mean():,.0f}이지만 중앙값은 {df['Total_Spending'].median():,.0f}로")
print(f"     왜도={sp_skew:.2f}, CV={sp_cv:.0f}%이며 고객 간 소비 편차가 매우 큽니다.")
print(f"   - 자녀 보유율 {df['Has_Children'].mean():.1%}, 캠페인 1건 이상 반응율 {df['Campaign_Response'].mean():.1%}")

print("\n2. 핵심 발견:")
print(f"   - Income: 평균({df['Income'].mean():,.0f})과 중앙값({df['Income'].median():,.0f})이 거의 같아")
print(f"     정규분포에 가깝습니다. 소득 기반 고객 분류에 적합한 변수입니다.")
print(f"   - Total_Spending: 강한 우측 비대칭으로, 소수 고소비 VIP와 다수 저소비 고객으로 양극화됩니다.")
print(f"   - 와인({df['MntWines'].mean():,.0f})이 총 소비의 {df['MntWines'].sum()/df['Total_Spending'].sum():.1%},")
print(f"     육류({df['MntMeatProducts'].mean():,.0f})가 {df['MntMeatProducts'].sum()/df['Total_Spending'].sum():.1%}로 두 카테고리가 매출의 핵심입니다.")
print(f"   - 소득 그룹 간 소비 격차가 뚜렷합니다 (바이올린 플롯에서 Low/Mid/High 분포 확인).")

print("\n3. 후속 분석 가설:")
print("   - H1: 자녀가 없는 고객이 자녀가 있는 고객보다 소비가 유의하게 높다. (Part 3)")
print("   - H2: 같은 고객이 와인과 육류에 쓰는 금액에 유의한 차이가 있다. (Part 3)")
print("   - H3: 교육 수준에 따라 소비 수준에 유의한 차이가 있다. (Part 4)")
print("   - H4: 이전 캠페인 반응 경험이 향후 캠페인 반응에 영향을 미친다. (Part 5)")

---
## Part 2: 확률분포 모델링 — "고객 행동은 어떤 패턴을 따르는가?"

실제 고객 데이터에 이론적 확률분포를 적합하여 고객 행동을 모델링합니다.

### 문제 2-1. 이항분포 — 캠페인 수락 모델링

마지막 캠페인(Response)의 수락률을 기반으로, 무작위 10명의 고객 중:

- **(a)** 정확히 2명이 수락할 확률
- **(b)** 3명 이상이 수락할 확률
- **(c)** 이항분포 PMF를 시각화하세요.

In [ ]:
p_response = df['Response'].mean()   # 캠페인 수락률을 p로 추정합니다
n_trial = 10                          # 10명에게 캠페인 발송 시나리오

prob_exactly_2 = stats.binom.pmf(2, n_trial, p_response)       # P(X = 2)
prob_3_or_more = 1 - stats.binom.cdf(2, n_trial, p_response)   # P(X ≥ 3) = 1 - P(X ≤ 2)

print("[문제 2-1] 이항분포 — 캠페인 수락 모델링")
print("=" * 60)
print(f"  캠페인 수락률 (p): {p_response:.4f}")
print(f"  시행 횟수 (n):    {n_trial}")
print(f"\n  (a) P(X = 2): {prob_exactly_2:.4f} ({prob_exactly_2:.2%})")
print(f"  (b) P(X ≥ 3): {prob_3_or_more:.4f} ({prob_3_or_more:.2%})")
print(f"\n  → 10명 중 정확히 2명이 수락할 확률은 {prob_exactly_2:.2%}입니다.")
print(f"  → 10명 중 3명 이상 수락할 확률은 {prob_3_or_more:.2%}로 매우 낮습니다.")

# (d) 실무 시나리오: 마케팅팀이 100명에게 캠페인을 발송합니다
n_campaign = 100
target_accept = 15

# 기대 수락자 수와 표준편차
expected = n_campaign * p_response
std_binom = np.sqrt(n_campaign * p_response * (1 - p_response))

# 15명 이상 수락할 확률
prob_target = 1 - stats.binom.cdf(target_accept - 1, n_campaign, p_response)

# 90% 확률로 달성 가능한 최소 수락 수
min_accept_90 = stats.binom.ppf(0.10, n_campaign, p_response)

print(f"\n  [실무 시나리오] 100명에게 캠페인 발송 시:")
print(f"    기대 수락자 수: {expected:.1f}명 (± {std_binom:.1f})")
print(f"    15명 이상 수락할 확률: {prob_target:.4f} ({prob_target:.1%})")
print(f"    90% 확률로 최소 {int(min_accept_90)}명 이상 수락이 보장됩니다.")
print(f"    → 마케팅팀이 최소 15명 수락을 목표로 한다면, 수락률 {p_response:.1%}로는")
if prob_target >= 0.5:
    print(f"      달성 확률이 {prob_target:.1%}로 현실적인 목표입니다.")
else:
    print(f"      달성 확률이 {prob_target:.1%}로, 발송 대상을 늘리거나 수락률 개선이 필요합니다.")

In [ ]:
# (c) 이항분포 PMF 시각화 — 색상으로 P(X=2)와 P(X≥3) 영역을 구분합니다
x = np.arange(0, n_trial + 1)
pmf = stats.binom.pmf(x, n_trial, p_response)
# 각 막대에 색상 부여: k=2 → amber, k≥3 → rose, 나머지 → blue
colors = [COLORS['amber'] if k == 2 else COLORS['rose'] if k >= 3
          else COLORS['blue'] for k in x]

fig, ax = plt.subplots(figsize=(10, 5))
sns.barplot(x=x, y=pmf, hue=x, palette=colors, legend=False, ax=ax)
for i, v in enumerate(pmf):
    if v > 0.005:
        ax.text(i, v + 0.005, f'{v:.3f}', ha='center', fontsize=9)
ax.set(xlabel='수락 고객 수 (k)', ylabel='확률 P(X=k)',
       title=f'이항분포 B(n={n_trial}, p={p_response:.3f})')
plt.tight_layout()
plt.show()

### 문제 2-2. 포아송 분포 — 웹사이트 방문 모델링

`NumWebVisitsMonth`(월간 웹 방문 수)에 포아송 분포를 적합합니다.

- **(a)** λ(람다) 추정
- **(b)** 실측 PMF와 이론 PMF 비교 막대그래프

In [ ]:
web_visits = df['NumWebVisitsMonth']
lambda_hat = web_visits.mean()  # λ 추정: 표본 평균 = MLE

x_pois = np.arange(0, web_visits.max() + 1)
observed_freq = web_visits.value_counts().sort_index()
observed_pmf = observed_freq / len(web_visits)              # 실측 PMF
theoretical_pmf = stats.poisson.pmf(x_pois, lambda_hat)    # 이론 PMF

print("[문제 2-2] 포아송 분포 — 웹사이트 방문 모델링")
print("=" * 60)
print(f"  λ(평균 방문 수): {lambda_hat:.3f}")
print(f"  분산: {web_visits.var():.3f}")
ratio = web_visits.var() / lambda_hat
print(f"  → 분산/평균 비율: {ratio:.3f}")
if abs(ratio - 1) < 0.3:
    print(f"  → 비율이 1에 가까워({ratio:.3f}) 포아송 분포의 '평균=분산' 가정에 부합합니다.")
    print("    다만 그래프를 보면 실측 분포의 최빈값이 이론 분포보다 약간 높은 쪽에 있어")
    print("    완벽한 적합은 아닙니다. 포아송 모델은 근사적으로 적절합니다.")
else:
    print(f"  → 비율이 1에서 벗어나({ratio:.3f}) 과분산(overdispersion)이 존재합니다.")
    print("    포아송 분포의 적합도가 제한적일 수 있습니다.")

# (c) 실무 시나리오: 웹 트래픽 관리 및 고객 세분화
heavy_threshold = 10
heavy_user_rate_actual = (web_visits >= heavy_threshold).mean()
heavy_user_rate_theory = 1 - stats.poisson.cdf(heavy_threshold - 1, lambda_hat)

# 서버 용량 계획: 상위 5% 고객의 방문 수
visit_95th_actual = web_visits.quantile(0.95)
visit_95th_theory = stats.poisson.ppf(0.95, lambda_hat)

print(f"\n  [실무 시나리오] 웹 트래픽 기반 고객 세분화:")
print(f"    '과다 방문' 기준 (월 {heavy_threshold}회 이상):")
print(f"      이론 비율: {heavy_user_rate_theory:.4f} ({heavy_user_rate_theory:.1%})")
print(f"      실측 비율: {heavy_user_rate_actual:.4f} ({heavy_user_rate_actual:.1%})")
print(f"    → 실측으로는 약 {heavy_user_rate_actual:.1%}만 과다 방문자입니다.")
print(f"    → 포아송 모델은 꼬리(극단값)를 과대 추정합니다 ({heavy_user_rate_theory:.1%} vs {heavy_user_rate_actual:.1%}).")
print(f"      이는 실제 분포가 포아송보다 꼬리가 짧기 때문이며, 실무에서는 실측 데이터를 우선해야 합니다.")
print(f"\n    서버 용량 계획 (상위 5% 수용):")
print(f"      이론: 월 {int(visit_95th_theory)}회 | 실측: 월 {int(visit_95th_actual)}회")
print(f"    → 서버는 월 {int(visit_95th_actual)}회 이상의 접속을 수용해야 합니다.")

In [ ]:
# 실측 PMF vs 이론 PMF 비교 — 나란히 놓인 막대그래프로 적합도를 시각적으로 확인합니다
obs_vals = [observed_pmf.get(k, 0) for k in x_pois]
compare_df = pd.DataFrame({
    '웹 방문 수': np.tile(x_pois, 2),
    '확률': np.concatenate([obs_vals, theoretical_pmf]),
    '유형': ['실측 PMF'] * len(x_pois) + [f'포아송 PMF (λ={lambda_hat:.2f})'] * len(x_pois)
})

fig, ax = plt.subplots(figsize=(10, 5))
sns.barplot(data=compare_df, x='웹 방문 수', y='확률', hue='유형',
            palette=[COLORS['blue'], COLORS['rose']], alpha=0.7, edgecolor='white', ax=ax)
ax.set(title='웹 방문 수: 실측 vs 포아송 분포')
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

### 문제 2-3. 정규분포 — 소득 모델링

`Income`에 정규분포를 적합합니다.
Part 1에서 Income의 왜도가 작고 평균≈중앙값임을 확인했습니다. 정규분포가 잘 맞을까요?

- **(a)** μ, σ 추정 및 CDF/PPF 활용 — 이론 확률과 실측 확률 비교
- **(b)** 히스토그램 + 정규 PDF 겹쳐 그리기

In [ ]:
mu_income = df['Income'].mean()       # μ 추정: 표본 평균
sigma_income = df['Income'].std()     # σ 추정: 표본 표준편차

# CDF 활용: 특정 구간에 속할 확률 계산
prob_below_30k = stats.norm.cdf(30000, mu_income, sigma_income)      # P(X < 30000)
prob_above_80k = 1 - stats.norm.cdf(80000, mu_income, sigma_income)  # P(X > 80000)
# PPF 활용: "상위 10%에 해당하는 소득 기준선" 산출
income_90th = stats.norm.ppf(0.90, mu_income, sigma_income)          # 90번째 백분위수

actual_below_30k = (df['Income'] < 30000).mean()
actual_above_80k = (df['Income'] > 80000).mean()
actual_90th = df['Income'].quantile(0.90)

print("[문제 2-3] 정규분포 — 소득 모델링")
print("=" * 60)
print(f"  μ (평균 소득): {mu_income:,.2f}")
print(f"  σ (표준편차):  {sigma_income:,.2f}")
print(f"  왜도: {stats.skew(df['Income']):.4f} → 거의 대칭에 가까운 분포")
print(f"\n  CDF 활용 — 이론 확률 vs 실측 비율:")
print(f"    P(Income < 30,000): 이론 {prob_below_30k:.4f} vs 실측 {actual_below_30k:.4f}")
print(f"    P(Income > 80,000): 이론 {prob_above_80k:.4f} vs 실측 {actual_above_80k:.4f}")
print(f"\n  PPF 활용:")
print(f"    상위 10% 소득 기준: 이론 {income_90th:,.0f} vs 실측 {actual_90th:,.0f}")
print(f"\n  → 이론 확률과 실측 비율이 매우 유사합니다!")
print(f"    Income 분포가 정규분포에 잘 적합됨을 의미합니다.")

# (c) 실무 시나리오: 소득 기반 고객 등급 설계
vip_threshold = stats.norm.ppf(0.95, mu_income, sigma_income)
discount_threshold = stats.norm.ppf(0.10, mu_income, sigma_income)
vip_actual = df['Income'].quantile(0.95)
discount_actual = df['Income'].quantile(0.10)

print(f"\n  [실무 시나리오] 소득 기반 고객 등급 설계:")
print(f"    VIP (상위 5%): 소득 ≥ {vip_threshold:,.0f} (이론) | {vip_actual:,.0f} (실측)")
print(f"    할인 대상 (하위 10%): 소득 ≤ {discount_threshold:,.0f} (이론) | {discount_actual:,.0f} (실측)")
print(f"    → VIP 고객 기준선을 소득 {vip_actual:,.0f} 이상으로 설정하면")
print(f"      약 {(df['Income'] >= vip_actual).sum():,}명이 프리미엄 서비스 대상이 됩니다.")

In [ ]:
# 히스토그램 + 이론 정규 PDF 겹쳐 그리기 — 적합도를 시각적으로 확인합니다
fig, ax = plt.subplots(figsize=(10, 5))
sns.histplot(df['Income'], bins=40, stat='density', color=COLORS['blue'],
             edgecolor='white', alpha=0.5, label='실측 분포', ax=ax)

x_norm = np.linspace(df['Income'].min(), df['Income'].max(), 300)
ax.plot(x_norm, stats.norm.pdf(x_norm, mu_income, sigma_income),
        color=COLORS['red'], lw=2.5,
        label=f'정규분포 N({mu_income:,.0f}, {sigma_income:,.0f}²)')
ax.axvline(mu_income, color=COLORS['amber'], ls='--', lw=1.5,
           label=f'μ = {mu_income:,.0f}')
ax.axvline(income_90th, color=COLORS['emerald'], ls=':', lw=1.5,
           label=f'상위 10%: {income_90th:,.0f}')
ax.set(xlabel='Income', ylabel='밀도', title='Income 분포와 정규분포 적합')
ax.legend(fontsize=10)
plt.tight_layout()
plt.show()

---
## Part 3: 추정과 가설검정 — "고소득 고객과 저소득 고객의 소비는 다른가?"

신뢰구간과 가설검정을 통해 고객 그룹 간 차이를 통계적으로 검증합니다.

### 문제 3-1. 신뢰구간 추정

- **(a)** Total_Spending 평균의 90%, 95%, 99% 신뢰구간을 계산하세요.
- **(b)** Income_Group별 95% CI를 계산하고 오차막대 그래프로 비교하세요.

In [ ]:
print("[문제 3-1] 신뢰구간 추정")
print("=" * 60)

data_spending = df['Total_Spending']
n = len(data_spending)
mean_sp = data_spending.mean()
se_sp = data_spending.std(ddof=1) / np.sqrt(n)  # 표준오차(SE) = s / √n

print("(a) Total_Spending 평균의 신뢰구간:")
for conf, alpha in [(0.90, 0.10), (0.95, 0.05), (0.99, 0.01)]:
    t_crit = stats.t.ppf(1 - alpha/2, df=n-1)  # t-분포 임계값
    margin = t_crit * se_sp                       # 오차한계 = t × SE
    ci_lower = mean_sp - margin
    ci_upper = mean_sp + margin
    print(f"    {conf:.0%} CI: [{ci_lower:,.2f}, {ci_upper:,.2f}]  (오차한계: ±{margin:,.2f})")


print(f"\n    표본평균: {mean_sp:,.2f}, SE: {se_sp:,.2f}, n: {n}")

In [ ]:
data = df['Total_Spending']
n = len(data)
mean_sp = data.mean()
se_sp = data.std() / np.sqrt(n) #표준 오차
se_sp = stats.sem(data)

print('평균:', mean_sp)
print('표준오차:', se_sp)
for conf, alpha in [(0.9, 0.1), (0.95, 0.05), (0.99, 0.01)] :
    t_crit = stats.t.ppf(1-alpha/2, df=n-1)
    margin = t_crit * se_sp
    ci_lower = mean_sp - margin
    ci_upper = mean_sp + margin
    print(f'{alpha:.2f} CI:[{ci_lower},{ci_upper}]')
    print(stats.t.interval(conf, df=n-1, loc=mean_sp, scale=se_sp))

In [ ]:
# (b) Income_Group별 95% CI 오차막대 그래프
print("\n(b) Income_Group별 95% CI:")

groups = ['Low', 'Mid', 'High']
group_means = []
group_cis = []
group_colors_list = [COLORS['sky'], COLORS['amber'], COLORS['emerald']]

for g in groups:
    gdata = df[df['Income_Group'] == g]['Total_Spending']
    gn = len(gdata)
    gmean = gdata.mean()
    gse = gdata.std(ddof=1) / np.sqrt(gn)
    t_crit = stats.t.ppf(0.975, df=gn-1)
    margin = t_crit * gse
    group_means.append(gmean)
    group_cis.append(margin)
    print(f"    {g:>4}: 평균={gmean:>8,.1f}, 95% CI=[{gmean-margin:>8,.1f}, {gmean+margin:>8,.1f}], n={gn}")

# sns.barplot은 자동으로 평균 + 95% CI 오차막대를 그려줍니다
fig, ax = plt.subplots(figsize=(8, 5))
sns.barplot(data=df, x='Income_Group', y='Total_Spending',
            order=['Low', 'Mid', 'High'], errorbar=('ci', 95),
            palette=[COLORS['sky'], COLORS['amber'], COLORS['emerald']],
            edgecolor='white', alpha=0.7, ax=ax)
# 각 막대 위에 평균값 표시
for i, (mean, ci) in enumerate(zip(group_means, group_cis)):
    ax.text(i, mean + ci + 10, f'{mean:,.0f}',
            ha='center', fontsize=11, fontweight='bold')
ax.set(xlabel='Income Group', ylabel='Total Spending',
       title='소득 그룹별 평균 소비 (±95% CI)')
plt.tight_layout()
plt.show()

ci_overlap = group_means[0] + group_cis[0] < group_means[1] - group_cis[1]
if ci_overlap:
    print("\n→ 세 그룹의 95% 신뢰구간이 겹치지 않습니다.")
    print("  이는 소득 그룹 간 평균 소비 차이가 통계적으로 유의할 가능성이 높음을 시사합니다.")
else:
    print("\n→ 일부 그룹의 95% 신뢰구간이 겹칩니다.")
    print("  정확한 판단을 위해서는 Part 4에서 ANOVA/Kruskal-Wallis 검정이 필요합니다.")

In [ ]:
# TODO 3-1(b): Income_Group별 95% CI 계산 + 오차막대 그래프

groups = ['Low', 'Mid', "High"]
for g in groups :
    gdata = df[df['Income_Group']==g]['Total_Spending']
    gn = len(gdata)
    print(stats.t.interval(0.95, df=gn-1, loc=gdata.mean(), scale=stats.sem(gdata)))


sns.barplot(data=df, x='Income_Group', y='Total_Spending', order=groups, errorbar=('ci', 95),)
plt.show()

### 문제 3-2. 독립표본 t-검정 (Welch's t-test)

**가설**: 자녀가 없는 고객의 총 소비가 자녀가 있는 고객보다 높은가?

- H₀: μ_무자녀 = μ_유자녀
- H₁: μ_무자녀 ≠ μ_유자녀

In [ ]:
no_child = df[~df['Has_Children']]['Total_Spending']
yes_child = df[df['Has_Children']]['Total_Spending']

t_stat, p_value = stats.ttest_ind(no_child, yes_child, equal_var=False)
cohens_d = (no_child.mean() - yes_child.mean()) / np.sqrt(
    (no_child.var(ddof=1) + yes_child.var(ddof=1)) / 2
)

print("[문제 3-2] 독립표본 t-검정 (자녀 유무별 소비)")
print("=" * 60)
print(f"  무자녀 그룹: n={len(no_child)}, 평균={no_child.mean():,.1f}, SD={no_child.std():,.1f}")
print(f"  유자녀 그룹: n={len(yes_child)}, 평균={yes_child.mean():,.1f}, SD={yes_child.std():,.1f}")
print(f"\n  Welch's t-검정:")
print(f"    t = {t_stat:.4f}")
print(f"    p = {p_value:.6f}")
print(f"    Cohen's d = {cohens_d:.4f}")
print(f"    (기준: 0.2 미만 매우 작은, 0.2~0.5 작은, 0.5~0.8 중간, 0.8 이상 큰)")

d_label = '매우 작은' if abs(cohens_d) < 0.2 else ('작은' if abs(cohens_d) < 0.5 else ('중간' if abs(cohens_d) < 0.8 else '큰'))
print(f"\n  → p < 0.05이므로 귀무가설을 기각합니다." if p_value < 0.05
      else f"\n  → p ≥ 0.05이므로 귀무가설을 기각하지 못합니다.")
print(f"  → 효과 크기(Cohen's d = {abs(cohens_d):.3f})는 '{d_label}' 효과에 해당합니다.")
if abs(cohens_d) >= 0.8:
    print(f"  → 무자녀 고객의 평균 소비({no_child.mean():,.0f})가 유자녀 고객({yes_child.mean():,.0f})의")
    print(f"    약 {no_child.mean()/yes_child.mean():.1f}배에 달하며, 실질적으로도 매우 큰 차이입니다.")

In [ ]:
# 자녀 유무별 소비 비교 — sns.boxplot으로 간결하게 표현합니다
fig, ax = plt.subplots(figsize=(8, 5))
df['자녀유무'] = df['Has_Children'].map({False: '무자녀', True: '유자녀'})
sns.boxplot(data=df, x='자녀유무', y='Total_Spending',
            palette=[COLORS['blue'], COLORS['rose']], width=0.5, ax=ax)

sig_text = f"t={t_stat:.2f}, p={p_value:.4f}\nCohen's d={cohens_d:.3f}"
ax.annotate(sig_text, xy=(0.5, max(no_child.max(), yes_child.max()) * 0.9),
            fontsize=10, ha='center',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='lightyellow', alpha=0.8))
ax.set(ylabel='Total Spending', title='자녀 유무별 총 소비 비교')
df.drop(columns='자녀유무', inplace=True)  # 임시 컬럼 제거
plt.tight_layout()
plt.show()

### 문제 3-3. 대응표본 t-검정

같은 고객의 **와인 소비**(`MntWines`)와 **육류 소비**(`MntMeatProducts`)를 비교합니다.

- H₀: μ_Wine = μ_Meat (차이 = 0)
- H₁: μ_Wine ≠ μ_Meat

In [ ]:
wines = df['MntWines']
meat = df['MntMeatProducts']
diff = wines - meat  # 같은 고객의 와인-육류 소비 차이

t_paired, p_paired = stats.ttest_rel(wines, meat)
d_paired = diff.mean() / diff.std(ddof=1)  # 대응 Cohen's d

print("[문제 3-3] 대응표본 t-검정 (와인 vs 육류 소비)")
print("=" * 60)
print(f"  와인 평균:   {wines.mean():>8,.1f}")
print(f"  육류 평균:   {meat.mean():>8,.1f}")
print(f"  차이 평균:   {diff.mean():>8,.1f}")
print(f"  차이 SD:     {diff.std():>8,.1f}")
print(f"\n  대응표본 t-검정:")
print(f"    t = {t_paired:.4f}")
print(f"    p = {p_paired:.6f}")
d_paired_label = '매우 작은' if abs(d_paired) < 0.2 else ('작은' if abs(d_paired) < 0.5 else ('중간' if abs(d_paired) < 0.8 else '큰'))
print(f"    Cohen's d = {d_paired:.4f}")
print(f"    (기준: 0.2 미만 매우 작은, 0.2~0.5 작은, 0.5~0.8 중간, 0.8 이상 큰)")
print(f"\n  → p < 0.05이므로 귀무가설을 기각합니다." if p_paired < 0.05
      else f"\n  → p ≥ 0.05이므로 귀무가설을 기각하지 못합니다.")
print(f"  → 고객들은 평균적으로 육류보다 와인에 {abs(diff.mean()):,.1f} 더 {'많이' if diff.mean()>0 else '적게'} 소비합니다.")
print(f"  → Cohen's d = {abs(d_paired):.3f}로 '{d_paired_label}' 효과입니다.")
print(f"    와인이 이 기업의 핵심 매출 카테고리임을 확인할 수 있습니다.")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# (좌) 대응 차이 분포 히스토그램
sns.histplot(diff, bins=30, color=COLORS['indigo'], edgecolor='white', alpha=0.6, ax=axes[0])
axes[0].axvline(0, color=COLORS['red'], ls='--', lw=2, label='차이 = 0')
axes[0].axvline(diff.mean(), color=COLORS['amber'], lw=2,
                label=f'평균 차이: {diff.mean():,.1f}')
axes[0].set(xlabel='와인 - 육류 소비 차이', ylabel='빈도', title='대응 차이 분포')
axes[0].legend(fontsize=10)

# (우) 카테고리별 평균 비교 — melt 후 sns.barplot 사용
melt_df = df[['MntWines', 'MntMeatProducts']].melt(var_name='카테고리', value_name='소비액')
melt_df['카테고리'] = melt_df['카테고리'].map({'MntWines': '와인', 'MntMeatProducts': '육류'})
sns.barplot(data=melt_df, x='카테고리', y='소비액', errorbar='se',
            palette=[COLORS['violet'], COLORS['teal']], edgecolor='white', alpha=0.7, ax=axes[1])
for i, mean in enumerate([wines.mean(), meat.mean()]):
    axes[1].text(i, mean + 5, f'{mean:,.1f}', ha='center', fontsize=11, fontweight='bold')
axes[1].set(ylabel='평균 소비액', title='와인 vs 육류 평균 소비 (±SEM)')

plt.tight_layout()
plt.show()

### 문제 3-4. 검정력 분석

- **(a)** 문제 3-2의 효과 크기와 표본 크기를 사용하여 검정력(Power)을 계산하세요.
- **(b)** 작은 효과(d=0.2)를 탐지하기 위해 필요한 그룹당 표본 크기를 구하세요.

In [ ]:
power_analysis = TTestIndPower()
# (a) 기존 분석의 검정력 계산 — "이 실험으로 차이를 탐지할 능력이 충분했나?"
power = power_analysis.solve_power(
    effect_size=abs(cohens_d),        # 관측된 효과 크기
    nobs1=len(no_child),              # 그룹 1 표본 크기
    ratio=len(yes_child)/len(no_child),  # 그룹 2/그룹 1 비율
    alpha=0.05
)

# (b) 작은 효과(d=0.2)를 탐지하려면 그룹당 몇 명 필요한가?
n_needed = power_analysis.solve_power(
    effect_size=0.2,    # 작은 효과 크기
    power=0.8,          # 80% 검정력 목표
    ratio=1,            # 두 그룹 크기 동일
    alpha=0.05
)

print("[문제 3-4] 검정력 분석")
print("=" * 60)
print(f"  (a) 문제 3-2 기준 검정력:")
print(f"      효과 크기: d = {abs(cohens_d):.4f}")
print(f"      표본 크기: n1={len(no_child)}, n2={len(yes_child)}")
print(f"      α = 0.05")
print(f"      검정력(Power) = {power:.4f} ({power:.1%})")
if power >= 0.8:
    print(f"      → 검정력이 80% 이상으로 충분합니다.")
else:
    print(f"      → 검정력이 80% 미만으로 표본이 부족할 수 있습니다.")

print(f"\n  (b) 작은 효과(d=0.2) 탐지에 필요한 표본 크기:")
print(f"      필요 표본 크기: 그룹당 {int(np.ceil(n_needed))}명")
print(f"      → 총 {int(np.ceil(n_needed))*2}명이 필요합니다.")

---
## Part 4: 고급 검정 — "교육, 결혼, 캠페인 반응의 관계는?"

정규성 검정을 바탕으로 적절한 검정 방법을 선택하고,
범주형 변수 간 관계를 분석합니다.

### 문제 4-1. 정규성 검정

Education_Group별 Total_Spending의 정규성을 Shapiro-Wilk 검정과 Q-Q Plot으로 확인하세요.
결과에 따라 문제 4-2에서 모수/비모수 검정을 선택합니다.

In [ ]:
edu_groups = ['Undergraduate', 'Graduate', 'Postgraduate']

print("[문제 4-1] 정규성 검정 (Education_Group별 Total_Spending)")
print("=" * 60)

normality_results = {}
for g in edu_groups:
    gdata = df[df['Education_Group'] == g]['Total_Spending']
    stat_sw, p_sw = stats.shapiro(gdata)
    normality_results[g] = p_sw
    verdict = "정규성 유지" if p_sw >= 0.05 else "정규성 기각"
    print(f"  {g:>15}: W={stat_sw:.4f}, p={p_sw:.6f} → {verdict}")

all_normal = all(p >= 0.05 for p in normality_results.values())
group_sizes = {g: len(df[df['Education_Group'] == g]) for g in edu_groups}
min_n = min(group_sizes.values())

if all_normal:
    print(f"\n  → 모든 그룹이 정규성을 만족합니다. 모수 검정(ANOVA)을 사용합니다.")
else:
    print(f"\n  → Shapiro-Wilk 검정에서 정규성이 기각되었습니다 (p ≈ 0).")
    print(f"    이는 Total_Spending의 강한 우측 비대칭(왜도={stats.skew(df['Total_Spending']):.2f}) 때문입니다.")
    print(f"\n  ⚠️ 그러나 각 그룹의 표본 크기가 충분히 큽니다:")
    for g in edu_groups:
        print(f"      {g:>15}: n = {group_sizes[g]}")
    print(f"\n    중심극한정리(CLT)에 의해 표본이 클 때(n ≥ 30) 표본 평균의 분포는")
    print(f"    정규분포에 근사하므로, ANOVA는 정규성 위반에 강건(robust)합니다.")
    print(f"    또한 Shapiro-Wilk 검정은 표본이 클수록 사소한 편차에도 기각하는 경향이 있습니다.")
    print(f"    → 최소 그룹 n={min_n} ≥ 30이므로 정규성 위반은 문제가 되지 않습니다.")

# 등분산 검정 (Levene's test) — CLT와 무관하므로 반드시 확인
levene_stat, levene_p = stats.levene(*[df[df['Education_Group'] == g]['Total_Spending'] for g in edu_groups])
print(f"\n  등분산 검정 (Levene's Test):")
print(f"    W = {levene_stat:.4f}, p = {levene_p:.6f}")

if levene_p >= 0.05:
    print(f"    → p ≥ 0.05이므로 등분산 가정을 만족합니다.")
    print(f"    → One-way ANOVA를 사용합니다.")
    use_welch = False
else:
    print(f"    → p < 0.05이므로 등분산 가정이 위반됩니다.")
    print(f"    ⚠️ 등분산성 위반은 CLT와 무관합니다 — 표본이 아무리 커도 해결되지 않습니다.")
    print(f"    → Welch's ANOVA를 사용합니다.")
    use_welch = True

In [ ]:
for g in edu_groups :
    gdata = df[df['Education_Group']==g]['Total_Spending']
    stat, p = stats.shapiro(gdata)
    print(g, p, len(gdata)) #p값이 0.05보다 커야 정규성을 만족하다고 해석

# TODO 4-1(b): Levene 등분산 검정 → use_welch = True/False 설정
stat, p = stats.levene(*[df[df['Education_Group']==g]['Total_Spending'] for g in edu_groups])
print(p) #등분산성 가정을 위한 p> 0.05보다 커야 등분산성을 만족한다고 해석

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16,5))
for ax, g in zip(axes, edu_groups) :
    gdata = df[df['Education_Group']==g]['Total_Spending']
    stats.probplot(gdata, dist='norm', plot=ax)

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
qq_colors = [COLORS['blue'], COLORS['violet'], COLORS['teal']]

for ax, g, color in zip(axes, edu_groups, qq_colors):
    gdata = df[df['Education_Group'] == g]['Total_Spending']
    stats.probplot(gdata, dist="norm", plot=ax)
    ax.get_lines()[0].set_markerfacecolor(color)
    ax.get_lines()[0].set_markeredgecolor(color)
    ax.get_lines()[0].set_markersize(4)
    ax.get_lines()[1].set_color(COLORS['red'])
    p_val = normality_results[g]
    ax.set_title(f'{g}\n(Shapiro p={p_val:.4f})', fontsize=12, fontweight='bold')
    ax.grid(alpha=0.3)

plt.suptitle('Education Group별 Q-Q Plot', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### 문제 4-2. ANOVA + 사후검정

문제 4-1의 정규성/등분산 검정 결과에 따라 적절한 검정을 수행합니다.

- 등분산 만족 → One-way ANOVA + Tukey HSD + Cohen's d
- 등분산 위반 → Welch's ANOVA + Games-Howell + Hedges' g
- 효과 크기: η² (eta-squared) + ω² (omega-squared)

In [ ]:
print("[문제 4-2] 그룹 간 차이 검정")
print("=" * 60)

group_data = {g: df[df['Education_Group'] == g]['Total_Spending'].values for g in edu_groups}

# Levene 결과에 따라 ANOVA / Welch's ANOVA 선택
if use_welch:
    # Welch's ANOVA (scipy의 alexander_govern 또는 수동 계산)
    # scipy >= 1.7: stats.alexandergovern 사용 가능하지만, 범용성을 위해 f_oneway 후 보정 설명
    f_stat, p_anova = stats.f_oneway(*group_data.values())
    print(f"  Welch's ANOVA (등분산 위반 → Welch 보정):")
    print(f"    F = {f_stat:.4f}, p = {p_anova:.6f}")
    print(f"    ※ scipy.stats.f_oneway는 내부적으로 등분산을 가정하지만,")
    print(f"      큰 표본에서는 결과가 유사합니다. 엄밀한 Welch 보정을 원하면")
    print(f"      pingouin.welch_anova()를 사용할 수 있습니다.")
else:
    f_stat, p_anova = stats.f_oneway(*group_data.values())
    print(f"  One-way ANOVA (정규성: CLT 근거, 등분산: Levene 만족):")
    print(f"    F = {f_stat:.4f}, p = {p_anova:.6f}")
test_p = p_anova

# 참고: 비모수 검정 결과와 비교
h_stat, p_kw = stats.kruskal(*group_data.values())
print(f"\n  [참고] Kruskal-Wallis (비모수 대안):")
print(f"    H = {h_stat:.4f}, p = {p_kw:.6f}")
print(f"    → 두 검정 모두 {'유의' if p_anova < 0.05 and p_kw < 0.05 else '비유의'}한 결과로 일관됩니다.")

# η² (eta-squared) 계산
grand_mean = df['Total_Spending'].mean()
ss_between = sum(len(group_data[g]) * (group_data[g].mean() - grand_mean)**2 for g in edu_groups)
ss_total = sum((df['Total_Spending'] - grand_mean)**2)
eta_squared = ss_between / ss_total

N = len(df)
k_groups = len(edu_groups)
ss_within = ss_total - ss_between
ms_within = ss_within / (N - k_groups)

# ω² (omega-squared) — η²의 편향을 보정한 효과 크기
omega_squared = (ss_between - (k_groups - 1) * ms_within) / (ss_total + ms_within)

eta_label = '무시할 수준' if eta_squared < 0.01 else ('작은' if eta_squared < 0.06 else ('중간' if eta_squared < 0.14 else '큰'))
omega_label = '무시할 수준' if omega_squared < 0.01 else ('작은' if omega_squared < 0.06 else ('중간' if omega_squared < 0.14 else '큰'))
print(f"\n  효과 크기:")
print(f"    η² (eta-squared) = {eta_squared:.4f} → '{eta_label}' 효과")
print(f"    ω² (omega-squared) = {omega_squared:.4f} → '{omega_label}' 효과")
print(f"    (기준: 0.01 작은, 0.06 중간, 0.14 큰)")
print(f"    ※ ω²는 η²의 상향 편향을 보정한 값으로, 더 보수적인 추정치입니다.")

if test_p < 0.05:
    print(f"\n  → p < 0.05이므로 귀무가설을 기각합니다.")
    print(f"  → Education Group 간 소비에 유의한 차이가 있습니다.")
    for g in edu_groups:
        print(f"    {g:>15}: 평균={group_data[g].mean():>8,.1f}, 중앙값={np.median(group_data[g]):>8,.1f}")
    print(f"\n  → Undergraduate({group_data['Undergraduate'].mean():,.0f})가 Graduate({group_data['Graduate'].mean():,.0f}),")
    print(f"    Postgraduate({group_data['Postgraduate'].mean():,.0f})보다 낮습니다.")
    print(f"    다만 효과 크기(η²={eta_squared:.4f}, ω²={omega_squared:.4f})가 작아,")
    print(f"    교육 수준이 소비의 주된 결정요인은 아닙니다.")
else:
    print(f"\n  → p ≥ 0.05이므로 귀무가설을 기각하지 못합니다.")

In [ ]:
group_data = {g: df[df['Education_Group'] == g]['Total_Spending'].values for g in edu_groups}

# TODO 4-2(a): use_welch에 따라 ANOVA / Welch's ANOVA 수행 (Kruskal-Wallis도 병행)
import pingouin as pg

result = pg.welch_anova(data=df, dv='Total_Spending', between='Education_Group')
print(result)

# TODO 4-2(b): η², ω² 계산 — ω² = (SS_between - (k-1)*MS_within) / (SS_total + MS_within)
aov = pg.anova(data=df, dv='Total_Spending', between='Education_Group', detailed=True)
display(aov)

ss_b = aov['SS'][0] #SS_between 
df_b = aov['DF'][0] #(k-1)
ms_w = aov['MS'][1] #MS_within
ss_total = aov['SS'].sum()

omega_square = (ss_b - df_b *ms_w) /(ss_total + ms_w)
print(f"{omega_square:.3f}")


In [ ]:
# 사후검정: 등분산 만족 → Tukey HSD / 등분산 위반 → Games-Howell
if use_welch:
    print("사후검정 (Games-Howell) — 등분산 위반 시 적절한 사후검정:")
    print("-" * 60)
    gh = pg.pairwise_gameshowell(
        data=df, dv='Total_Spending', between='Education_Group'
    )
    print(gh[['A', 'B', 'diff', 'se', 'T', 'pval', 'hedges']].to_string(index=False))

    # 사후검정 해석 + 쌍별 Hedges' g
    print("\n사후검정 해석:")
    for _, row in gh.iterrows():
        g1, g2 = row['A'], row['B']
        mean_diff = row['diff']
        p_val = row['pval']
        hg = row['hedges']
        sig = "유의한 차이 있음" if p_val < 0.05 else "유의한 차이 없음"
        hg_label = '매우 작은' if abs(hg) < 0.2 else ('작은' if abs(hg) < 0.5 else ('중간' if abs(hg) < 0.8 else '큰'))

        print(f"  {g1} vs {g2}:")
        print(f"    평균 차이={mean_diff:+,.1f}, p={p_val:.4f} → {sig}")
        print(f"    Hedges' g={abs(hg):.3f} → '{hg_label}' 효과")

    print(f"\n  Hedges' g 기준: 0.2 미만 매우 작은, 0.2~0.5 작은, 0.5~0.8 중간, 0.8 이상 큰")
    print(f"  ※ Hedges' g는 Cohen's d의 소표본 편향을 보정한 버전입니다 (대표본에서 d ≈ g).")
else:
    print("사후검정 (Tukey HSD) — 등분산 만족 시 적절한 사후검정:")
    print("-" * 60)
    tukey = pairwise_tukeyhsd(
        df['Total_Spending'],
        df['Education_Group'],
        alpha=0.05
    )
    print(tukey.summary())

    # 사후검정 해석 + 쌍별 Cohen's d
    groups_u = tukey.groupsunique
    pair_idx = [(0,1), (0,2), (1,2)]
    print("\n사후검정 해석:")
    for idx, (i, j) in enumerate(pair_idx):
        g1, g2 = groups_u[i], groups_u[j]
        mean_diff = tukey.meandiffs[idx]
        p_val = tukey.pvalues[idx]
        sig = "유의한 차이 있음" if tukey.reject[idx] else "유의한 차이 없음"

        d1, d2 = group_data[g1], group_data[g2]
        n1, n2 = len(d1), len(d2)
        pooled_sd = np.sqrt(((n1 - 1) * d1.std(ddof=1)**2 + (n2 - 1) * d2.std(ddof=1)**2) / (n1 + n2 - 2))
        cohens_d = abs(d1.mean() - d2.mean()) / pooled_sd
        d_label = '매우 작은' if cohens_d < 0.2 else ('작은' if cohens_d < 0.5 else ('중간' if cohens_d < 0.8 else '큰'))

        print(f"  {g1} vs {g2}:")
        print(f"    평균 차이={mean_diff:+,.1f}, p={p_val:.4f} → {sig}")
        print(f"    Cohen's d={cohens_d:.3f} → '{d_label}' 효과")

    print(f"\n  Cohen's d 기준: 0.2 미만 매우 작은, 0.2~0.5 작은, 0.5~0.8 중간, 0.8 이상 큰")

print(f"  → 전체 η²/ω²보다 쌍별 효과크기가 실질적 차이를 더 잘 보여줍니다.")
print(f"  → Graduate와 Postgraduate 간에는 유의한 차이가 없으나,")
print(f"    Undergraduate는 다른 두 그룹보다 유의하게 낮은 소비를 보입니다.")

In [ ]:
# 시각화: sns.stripplot(개별 점) + sns.pointplot(평균) 조합
fig, ax = plt.subplots(figsize=(10, 6))
sns.stripplot(data=df, x='Education_Group', y='Total_Spending',
              order=edu_groups, palette=qq_colors, alpha=0.15, size=3, jitter=True, ax=ax)
sns.pointplot(data=df, x='Education_Group', y='Total_Spending',
              order=edu_groups, palette=qq_colors, markers='D', scale=1.2,
              errorbar=None, ax=ax)

sig_label = f"p={test_p:.4f}, η²={eta_squared:.4f}, ω²={omega_squared:.4f}"
ax.set(xlabel='Education Group', ylabel='Total Spending',
       title=f'교육 수준별 소비 비교 ({sig_label})')
plt.tight_layout()
plt.show()

### 문제 4-3. 카이제곱 독립성 검정

**가설**: 결혼 상태(Marital_Group)와 캠페인 반응(Campaign_Response)은 독립인가?

- H₀: Marital_Group과 Campaign_Response는 독립이다.
- H₁: 두 변수는 독립이 아니다.

In [ ]:
crosstab = pd.crosstab(df['Marital_Group'], df['Campaign_Response'],
                        margins=True, margins_name='합계')
print("[문제 4-3] 카이제곱 독립성 검정")
print("=" * 60)
print("\n교차표:")
print(crosstab)

crosstab_no_margin = pd.crosstab(df['Marital_Group'], df['Campaign_Response'])
chi2, p_chi, dof, expected = stats.chi2_contingency(crosstab_no_margin)

# Cochran 규칙 확인: 기대빈도 < 5인 셀이 전체의 20% 이하여야 함
n_cells = expected.size
n_low_expected = np.sum(expected < 5)
cochran_ok = (n_low_expected / n_cells) <= 0.20
print(f"\n  Cochran 규칙 확인:")
print(f"    기대빈도 < 5인 셀: {n_low_expected}개 / 전체 {n_cells}개 ({n_low_expected/n_cells:.0%})")
if cochran_ok:
    print(f"    → Cochran 규칙을 만족합니다. 카이제곱 검정이 적절합니다.")
else:
    print(f"    → Cochran 규칙을 위반합니다. Fisher 정확검정을 고려하세요.")

# φ (phi) — 2×2 교차표에서의 효과크기 (Cramér's V와 수치 동일)
n_total = crosstab_no_margin.sum().sum()
phi = np.sqrt(chi2 / n_total)

print(f"\n  카이제곱 검정:")
print(f"    χ² = {chi2:.4f}")
print(f"    자유도 = {dof}")
print(f"    p = {p_chi:.6f}")
print(f"    φ (phi) = {phi:.4f}")
print(f"    (기준: 0.1 작은, 0.3 중간, 0.5 큰)")

phi_label = '무시할 수준' if phi < 0.1 else ('작은' if phi < 0.3 else ('중간' if phi < 0.5 else '큰'))

# 오즈비 (2×2 교차표)
# crosstab_no_margin: rows=Marital_Group(Single, Together), cols=Campaign_Response(False, True)
ct = crosstab_no_margin
a = ct.loc['Single', True]    # Single & 반응
b = ct.loc['Single', False]   # Single & 무반응
c = ct.loc['Together', True]  # Together & 반응
d = ct.loc['Together', False] # Together & 무반응
odds_ratio = (a * d) / (b * c)
log_or_se = np.sqrt(1/a + 1/b + 1/c + 1/d)
or_ci_lower = np.exp(np.log(odds_ratio) - 1.96 * log_or_se)
or_ci_upper = np.exp(np.log(odds_ratio) + 1.96 * log_or_se)

rate_single = df[df['Marital_Group'] == 'Single']['Campaign_Response'].mean()
rate_together = df[df['Marital_Group'] == 'Together']['Campaign_Response'].mean()

print(f"\n  → p < 0.05이므로 귀무가설을 기각합니다." if p_chi < 0.05
      else f"\n  → p ≥ 0.05이므로 귀무가설을 기각하지 못합니다.")
print(f"  → φ = {phi:.4f}로 '{phi_label}' 연관성입니다.")

print(f"\n  오즈비 (Odds Ratio):")
print(f"    OR = {odds_ratio:.3f}  (95% CI: [{or_ci_lower:.3f}, {or_ci_upper:.3f}])")
if odds_ratio > 1:
    print(f"    → Single 고객이 Together 고객 대비 캠페인에 반응할 오즈가 {odds_ratio:.2f}배입니다.")
else:
    print(f"    → Together 고객이 Single 고객 대비 캠페인에 반응할 오즈가 {1/odds_ratio:.2f}배입니다.")

if p_chi < 0.05:
    print(f"\n  방향성 해석:")
    print(f"    Single 캠페인 반응률: {rate_single:.1%}")
    print(f"    Together 캠페인 반응률: {rate_together:.1%}")
    if rate_single > rate_together:
        print(f"  → 싱글 고객이 파트너가 있는 고객보다 캠페인 반응률이 {rate_single - rate_together:.1%}p 더 높습니다.")
        print(f"    다만 φ={phi:.4f}로 연관성이 매우 약하므로 실무적 의미는 제한적입니다.")

In [ ]:
print("\n기대빈도표:")
expected_df = pd.DataFrame(expected, index=crosstab_no_margin.index,
                            columns=crosstab_no_margin.columns)
print(expected_df.round(1))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

crosstab_pct = pd.crosstab(df['Marital_Group'], df['Campaign_Response'], normalize='index')
crosstab_pct.plot(kind='bar', ax=axes[0],
                  color=[COLORS['slate'], COLORS['emerald']], alpha=0.7, edgecolor='white')
axes[0].set_title('결혼 상태별 캠페인 반응률', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Marital Group', fontsize=12)
axes[0].set_ylabel('비율', fontsize=12)
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)
axes[0].legend(['미반응', '반응'], fontsize=10)
axes[0].grid(alpha=0.3, axis='y')

residuals = (crosstab_no_margin.values - expected) / np.sqrt(expected)
sns.heatmap(residuals, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            xticklabels=crosstab_no_margin.columns,
            yticklabels=crosstab_no_margin.index, ax=axes[1])
axes[1].set_title('표준화 잔차', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

### 문제 4-4. 적합도 검정

3개 구매 채널(Web, Catalog, Store)의 총 구매 비율이 균등하게 분포하는지 검정합니다.

- H₀: 세 채널의 구매 비율이 동일하다 (1/3 : 1/3 : 1/3)
- H₁: 세 채널의 구매 비율이 동일하지 않다

In [ ]:
channel_totals = df[purchase_cols].sum()
channel_names = ['Web', 'Catalog', 'Store']
observed = channel_totals.values
expected_uniform = np.full(3, observed.sum() / 3)

chi2_gof, p_gof = stats.chisquare(observed, f_exp=expected_uniform)

observed_p = observed / observed.sum()
expected_p = expected_uniform / expected_uniform.sum()
cohens_w = np.sqrt(np.sum((observed_p - expected_p)**2 / expected_p))

print("[문제 4-4] 적합도 검정 (구매 채널 균등분포)")
print("=" * 60)
print(f"\n  구매 채널별 총 구매 건수:")
for name, obs, exp in zip(channel_names, observed, expected_uniform):
    print(f"    {name:>8}: 관측 {obs:>6,}건,  기대 {exp:>8,.1f}건  (비율: {obs/observed.sum():.1%})")

print(f"\n  카이제곱 적합도 검정:")
print(f"    χ² = {chi2_gof:.4f}")
print(f"    자유도 = 2")
print(f"    p = {p_gof:.6f}")
print(f"    Cohen's w = {cohens_w:.4f}")
print(f"    (기준: 0.1 작은, 0.3 중간, 0.5 큰)")

w_label = '무시할 수준' if cohens_w < 0.1 else ('작은' if cohens_w < 0.3 else ('중간' if cohens_w < 0.5 else '큰'))
print(f"\n  → p < 0.05이므로 귀무가설을 기각합니다." if p_gof < 0.05
      else f"\n  → p ≥ 0.05이므로 귀무가설을 기각하지 못합니다.")
print(f"  → Cohen's w = {cohens_w:.4f}로 '{w_label}' 효과입니다.")
if p_gof < 0.05:
    max_channel = channel_names[np.argmax(observed)]
    min_channel = channel_names[np.argmin(observed)]
    print(f"\n  비즈니스 해석:")
    print(f"    {max_channel} 구매가 {observed_p[np.argmax(observed)]:.1%}로 가장 높고,")
    print(f"    {min_channel} 구매가 {observed_p[np.argmin(observed)]:.1%}로 가장 낮습니다.")
    print(f"    오프라인 매장이 주요 구매 채널이며, 카탈로그는 상대적으로 활용도가 낮습니다.")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
x_pos = np.arange(3)
width = 0.35

ax.bar(x_pos - width/2, observed, width, color=COLORS['blue'],
       alpha=0.7, edgecolor='white', label='관측')
ax.bar(x_pos + width/2, expected_uniform, width, color=COLORS['rose'],
       alpha=0.7, edgecolor='white', label='기대 (균등)')

for i, (obs, exp) in enumerate(zip(observed, expected_uniform)):
    ax.text(i - width/2, obs + 50, f'{obs:,}', ha='center', fontsize=10)
    ax.text(i + width/2, exp + 50, f'{exp:,.0f}', ha='center', fontsize=10)

ax.set_xticks(x_pos)
ax.set_xticklabels(channel_names)
ax.set_xlabel('구매 채널', fontsize=12)
ax.set_ylabel('총 구매 건수', fontsize=12)
ax.set_title(f'구매 채널 적합도 검정 (p={p_gof:.4f})', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

---
## Part 5: 상관분석 / 비율 비교 — "변수 간 관계를 파악하고, 캠페인 효과를 검증할 수 있는가?"

변수 간 상관관계를 분석하고, A/B 테스트로 캠페인 효과를 검증합니다.

### 문제 5-1. 상관분석

6개 변수(`Income`, `Age`, `Total_Spending`, `Total_Purchases`, `NumWebVisitsMonth`, `Recency`)의
Pearson/Spearman 상관계수를 계산하고 히트맵으로 시각화하세요.

In [ ]:
corr_vars = ['Income', 'Age', 'Total_Spending', 'Total_Purchases',
             'NumWebVisitsMonth', 'Recency']
df_corr = df[corr_vars]

pearson_corr = df_corr.corr(method='pearson')    # 선형 상관
spearman_corr = df_corr.corr(method='spearman')  # 순위 상관
display(pearson_corr)
display(spearman_corr)

print("[문제 5-1] 상관분석")
print("=" * 60)

print("\nPearson 상관계수 (주요 쌍):")
pairs = [('Income', 'Total_Spending'), ('Income', 'Total_Purchases'),
         ('Total_Spending', 'Total_Purchases'), ('NumWebVisitsMonth', 'Total_Spending'),
         ('Age', 'Total_Spending'), ('Recency', 'Total_Spending')]

for v1, v2 in pairs:
    r, p = stats.pearsonr(df[v1], df[v2])
    sig = "***" if p < 0.001 else ("**" if p < 0.01 else ("*" if p < 0.05 else "n.s."))
    print(f"  {v1:>20} vs {v2:<20}: r={r:+.4f}, p={p:.4e} {sig}")

r_inc_sp, _ = stats.pearsonr(df['Income'], df['Total_Spending'])
r_web_sp, _ = stats.pearsonr(df['NumWebVisitsMonth'], df['Total_Spending'])
r_rec_sp, _ = stats.pearsonr(df['Recency'], df['Total_Spending'])

print(f"\n핵심 발견:")
print(f"  - Income vs Total_Spending (r={r_inc_sp:+.3f}): 강한 양의 상관 → 소득이 소비의 핵심 예측변수")
print(f"  - WebVisits vs Total_Spending (r={r_web_sp:+.3f}): 중간 음의 상관 → 웹 방문이 잦을수록 소비가 적음")
print(f"    (저소비 고객이 가격 비교를 위해 웹을 더 자주 방문할 가능성)")
print(f"  - Recency vs Total_Spending (r={r_rec_sp:+.3f}): 거의 무상관 → 최근 구매 시점은 소비액과 무관")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

mask = np.triu(np.ones_like(pearson_corr, dtype=bool), k=1)
sns.heatmap(pearson_corr, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, vmin=-1, vmax=1, ax=axes[0], square=True,
            linewidths=0.5, linecolor='white')
axes[0].set_title('Pearson 상관행렬', fontsize=13, fontweight='bold')

sns.heatmap(spearman_corr, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, vmin=-1, vmax=1, ax=axes[1], square=True,
            linewidths=0.5, linecolor='white')
axes[1].set_title('Spearman 상관행렬', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# 주요 상관관계 산점도 + 추세선 — sns.regplot이 회귀선+CI를 자동으로 그려줍니다
scatter_pairs = [
    ('Income', 'Total_Spending', COLORS['blue']),
    ('NumWebVisitsMonth', 'Total_Spending', COLORS['teal']),
    ('Total_Spending', 'Total_Purchases', COLORS['violet']),
]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for ax, (xvar, yvar, color) in zip(axes, scatter_pairs):
    sns.regplot(data=df, x=xvar, y=yvar, color=color,
                scatter_kws={'alpha': 0.3, 's': 10},
                line_kws={'color': COLORS['red'], 'lw': 2}, ci=95, ax=ax)
    r_val, _ = stats.pearsonr(df[xvar], df[yvar])
    ax.set_title(f'{xvar} vs {yvar}\n(r={r_val:+.3f})', fontsize=12, fontweight='bold')

plt.suptitle('주요 변수 산점도 + 추세선', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n산점도 해석:")
print("  - Income vs Total_Spending: 소득이 높을수록 소비가 뚜렷하게 증가합니다.")
print("    다만 고소득 구간에서 소비의 분산이 커지는 '부채꼴' 형태가 관찰됩니다.")
print("  - WebVisits vs Total_Spending: 웹 방문이 잦을수록 소비가 적은 경향이 있습니다.")
print("    저소비 고객이 가격 비교 등을 위해 웹을 더 자주 방문할 가능성이 있습니다.")
print("  - Total_Spending vs Total_Purchases: 강한 양의 관계로, 소비액과 구매 건수가 함께 증가합니다.")

### 문제 5-2. 두 그룹 비율 비교 (관찰 연구)

이전 캠페인 반응 경험 유무에 따라 마지막 캠페인(Response) 수락률에 차이가 있는지 검증합니다.

- **Group A**: AcceptedCmp1~5 합계 = 0 (이전 캠페인 무반응)
- **Group B**: AcceptedCmp1~5 합계 ≥ 1 (이전 캠페인 반응 경험)
- **지표**: Response (마지막 캠페인 수락률)
- **검정 방법**: z-비율검정 + Cohen's h + Wilson 신뢰구간

> **주의**: 이 분석은 A/B 테스트가 **아닙니다**.
> A/B 테스트는 연구자가 참가자를 **무작위로 배정** (Random Assignment)하는 실험입니다.
> 여기서는 고객이 과거 행동에 의해 스스로 그룹에 배정되었으므로 **관찰 연구**이며,
> 인과관계를 주장할 수 없습니다.

In [ ]:
cmp_cols = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']
df['Prior_Response'] = df[cmp_cols].sum(axis=1)

group_a = df[df['Prior_Response'] == 0]
group_b = df[df['Prior_Response'] >= 1]

n_a, n_b = len(group_a), len(group_b)
conv_a = group_a['Response'].sum()
conv_b = group_b['Response'].sum()
rate_a = conv_a / n_a
rate_b = conv_b / n_b

z_stat, p_ab = proportions_ztest([conv_a, conv_b], [n_a, n_b])
cohens_h = proportion_effectsize(rate_a, rate_b)

ci_a = proportion_confint(conv_a, n_a, alpha=0.05, method='wilson')
ci_b = proportion_confint(conv_b, n_b, alpha=0.05, method='wilson')

print("[문제 5-2] 두 그룹 비율 비교 (관찰 연구)")
print("=" * 60)
print(f"\n  Group A (무반응 이력): n={n_a}, 수락={conv_a}, 수락률={rate_a:.4f} ({rate_a:.2%})")
print(f"    Wilson 95% CI: [{ci_a[0]:.4f}, {ci_a[1]:.4f}]")
print(f"  Group B (반응 이력):   n={n_b}, 수락={conv_b}, 수락률={rate_b:.4f} ({rate_b:.2%})")
print(f"    Wilson 95% CI: [{ci_b[0]:.4f}, {ci_b[1]:.4f}]")
print(f"\n  수락률 차이: {rate_b - rate_a:+.4f} ({(rate_b - rate_a):+.2%}p)")
print(f"\n  z-비율검정:")
print(f"    z = {z_stat:.4f}")
print(f"    p = {p_ab:.6f}")
print(f"    Cohen's h = {abs(cohens_h):.4f}")
print(f"    (기준: 0.2 미만 매우 작은, 0.2~0.5 작은, 0.5~0.8 중간, 0.8 이상 큰)")

h_label = '매우 작은' if abs(cohens_h) < 0.2 else ('작은' if abs(cohens_h) < 0.5 else ('중간' if abs(cohens_h) < 0.8 else '큰'))
print(f"\n  → p < 0.05이므로 두 그룹의 수락률에 유의한 차이가 있습니다." if p_ab < 0.05
      else f"\n  → p ≥ 0.05이므로 두 그룹의 수락률에 유의한 차이가 없습니다.")
print(f"  → Cohen's h = {abs(cohens_h):.4f}로 '{h_label}' 효과입니다.")
if p_ab < 0.05 and rate_b > rate_a:
    print(f"\n  → 이전 캠페인 반응 경험이 있는 고객(Group B)의 수락률이 {rate_b:.1%}로")
    print(f"    무반응 고객(Group A, {rate_a:.1%})보다 {rate_b - rate_a:.1%}p나 높습니다.")
    print(f"    이는 매우 큰 차이이지만, 관찰 연구이므로 인과관계로 해석할 수 없습니다.")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# (1) 수락률 비교 + CI
rates = [rate_a, rate_b]
ci_lowers = [rate_a - ci_a[0], rate_b - ci_b[0]]
ci_uppers = [ci_a[1] - rate_a, ci_b[1] - rate_b]
bars = axes[0].bar(['Group A\n(무반응 이력)', 'Group B\n(반응 이력)'], rates,
                    yerr=[ci_lowers, ci_uppers], capsize=8,
                    color=[COLORS['slate'], COLORS['emerald']], alpha=0.7,
                    edgecolor='white', error_kw={'linewidth': 2})
for bar, rate in zip(bars, rates):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                 f'{rate:.1%}', ha='center', fontsize=12, fontweight='bold')

axes[0].set_ylabel('수락률 (Response Rate)', fontsize=12)
axes[0].set_title(f'캠페인 수락률 비교 (p={p_ab:.4f})', fontsize=13, fontweight='bold')
axes[0].grid(alpha=0.3, axis='y')

# (2) 그룹 크기 파이 차트
axes[1].pie([n_a, n_b], labels=[f'Group A\n(n={n_a})', f'Group B\n(n={n_b})'],
            colors=[COLORS['slate'], COLORS['emerald']], autopct='%1.1f%%',
            startangle=90, textprops={'fontsize': 11})
axes[1].set_title('그룹 크기', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

> ⚠️ **이 분석이 A/B 테스트가 아닌 이유**
>
> A/B 테스트의 본질은 '무작위 배정(Random Assignment)'입니다.
>
> - **A/B 테스트**: 연구자가 참가자를 무작위로 Control/Treatment에 배정 → 인과관계 추론 가능
> - **이 분석**: 고객이 과거 행동에 의해 스스로 그룹에 배정 → 관찰 연구, 인과관계 주장 불가
>
> **가능한 교란 변수(Confounders)**:
>
> - 이전 캠페인 반응 고객은 원래 충성도/소비가 높은 고객일 수 있습니다 (선택 편향)
> - 소득, 연령 등 인구통계 변수가 캠페인 반응과 동시에 관련될 수 있습니다
> - 캠페인 유형이나 시기에 따른 차이가 반영되어 있을 수 있습니다
>
> → '이전 반응 경험이 수락률을 높인다'가 아니라,
> '원래 반응 성향이 높은 고객이 계속 반응하는 것'일 수 있습니다.
> → 인과관계를 검증하려면 무작위 배정 실험(RCT)을 설계해야 합니다. (문제 5-3)

### 문제 5-3. 진정한 A/B 테스트 설계

관찰 연구의 한계를 극복하기 위한 실험 설계를 수행합니다.
새로운 캠페인이 기존 수락률 대비 **2%p** 개선을 가져올 수 있는지 탐지하기 위한
최소 표본 크기를 산정하세요.

In [ ]:
baseline_rate = df['Response'].mean()
target_rate = baseline_rate + 0.02
h_design = proportion_effectsize(baseline_rate, target_rate)

power_prop = NormalIndPower()
n_design = power_prop.solve_power(
    effect_size=abs(h_design),
    alpha=0.05,
    power=0.8,
    ratio=1,
    alternative='larger'  # 개선(증가) 방향만 탐지 → 단측 검정
)

print("[문제 5-3] 진정한 A/B 테스트 설계")
print("=" * 60)
print(f"\n  기존 수락률 (baseline): {baseline_rate:.4f} ({baseline_rate:.2%})")
print(f"  목표 수락률 (target):   {target_rate:.4f} ({target_rate:.2%})")
print(f"  기대 개선폭:            {0.02:.4f} (2%p)")
print(f"\n  설계 매개변수:")
print(f"    Cohen's h = {abs(h_design):.4f}")
print(f"    α = 0.05 (유의수준)")
print(f"    Power = 0.80 (검정력)")
print(f"\n  필요 표본 크기:")
print(f"    그룹당: {int(np.ceil(n_design)):,}명")
print(f"    총 필요: {int(np.ceil(n_design))*2:,}명")

total_needed = int(np.ceil(n_design)) * 2
print(f"\n  실험 설계 제안:")
print(f"    1. {int(np.ceil(n_design)):,}명을 Control (기존 캠페인), {int(np.ceil(n_design)):,}명을 Treatment (새 캠페인)에 무작위 배정합니다.")
print(f"    2. 단측 검정(개선 방향)으로 2%p 이상의 개선을 80% 확률로 탐지할 수 있습니다.")
print(f"    3. 현재 데이터({len(df):,}명)로는 총 {total_needed:,}명이 필요하므로 {'충분합니다' if len(df) >= total_needed else '부족합니다'}.")
if len(df) < total_needed:
    print(f"       현재 데이터의 {len(df)/total_needed:.1%}에 불과하여, 더 많은 고객 확보가 필요합니다.")
    print(f"    4. 대안: 5%p 개선을 목표로 하면 필요 표본 크기가 크게 줄어듭니다.")

---
## Challenge: 종합 분석 보고서

지금까지의 분석을 종합하여 FreshCart 마케팅팀에 제출할 보고서를 작성합니다.

### Challenge 1. 고객 그룹별 특성 분석

`Income_Group` × `Has_Children` 6개 조합에 대해 종합 분석표를 작성하세요.

In [ ]:
print("[Challenge 1] 고객 고객 그룹별 특성 분석")
print("=" * 60)

segments = df.groupby(['Income_Group', 'Has_Children']).agg(
    고객수=('ID', 'count'),
    평균소득=('Income', 'mean'),
    평균소비=('Total_Spending', 'mean'),
    중앙값소비=('Total_Spending', 'median'),
    평균구매수=('Total_Purchases', 'mean'),
    캠페인반응률=('Campaign_Response', 'mean'),
    와인비중=('MntWines', lambda x: x.sum() / df.loc[x.index, 'Total_Spending'].sum() if df.loc[x.index, 'Total_Spending'].sum() > 0 else 0),
    웹방문수=('NumWebVisitsMonth', 'mean'),
).round(2)

print(segments.to_string())

In [ ]:
# 그룹 시각화 — sns.barplot의 hue로 자녀유무를 자동 분리합니다
df['자녀유무'] = df['Has_Children'].map({False: '무자녀', True: '유자녀'})
chart_cfg = [
    ('Total_Spending', '평균 소비', '그룹별 평균 소비'),
    ('Campaign_Response', '캠페인 반응률', '그룹별 캠페인 반응률'),
]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for ax, (col, ylabel, title) in zip(axes[:2], chart_cfg):
    sns.barplot(data=df, x='Income_Group', y=col, hue='자녀유무',
                order=['Low', 'Mid', 'High'], hue_order=['무자녀', '유자녀'],
                palette=[COLORS['blue'], COLORS['rose']],
                edgecolor='white', alpha=0.7, ax=ax)
    ax.set(xlabel='Income Group', ylabel=ylabel, title=title)
    ax.legend(fontsize=10)

# (3) 고객 수 — countplot 사용
sns.countplot(data=df, x='Income_Group', hue='자녀유무',
              order=['Low', 'Mid', 'High'], hue_order=['무자녀', '유자녀'],
              palette=[COLORS['blue'], COLORS['rose']],
              edgecolor='white', alpha=0.7, ax=axes[2])
axes[2].set(xlabel='Income Group', ylabel='고객 수', title='그룹별 고객 수')
axes[2].legend(fontsize=10)

df.drop(columns='자녀유무', inplace=True)  # 임시 컬럼 제거
plt.suptitle('고객 그룹별 종합 특성', fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### Challenge 2. 비즈니스 제안서

분석 결과를 종합하여 마케팅팀에 제안서를 작성합니다.

In [ ]:
print("[Challenge 2] 비즈니스 제안서")
print("=" * 60)

# 핵심 타겟 그룹 식별
best_segment = df.groupby(['Income_Group', 'Has_Children']).agg(
    평균소비=('Total_Spending', 'mean'),
    반응률=('Campaign_Response', 'mean'),
    고객수=('ID', 'count')
).reset_index()
best_segment['점수'] = best_segment['반응률'] * best_segment['평균소비']
best_segment = best_segment.sort_values('점수', ascending=False)

print("\n1. 타겟 고객 (반응률 × 소비 기준 상위 그룹):")
for _, row in best_segment.head(3).iterrows():
    child_label = '유자녀' if row['Has_Children'] else '무자녀'
    print(f"   - {row['Income_Group']} 소득 / {child_label}: "
          f"반응률 {row['반응률']:.1%}, 평균소비 {row['평균소비']:,.0f}, "
          f"고객 {row['고객수']}명")

print("\n2. 채널 전략:")
channel_by_income = df.groupby('Income_Group')[purchase_cols].mean()
high_catalog = channel_by_income.loc['High', 'NumCatalogPurchases']
low_web = df[df['Income_Group'] == 'Low']['NumWebVisitsMonth'].mean()
print(f"   - 고소득: 카탈로그 구매 평균 {high_catalog:.1f}건 → 프리미엄 카탈로그 마케팅 강화")
print(f"   - 저소득: 웹 방문 평균 {low_web:.1f}회/월 → 온라인 프로모션 집중")
store_ratio = df['NumStorePurchases'].sum() / df[purchase_cols].sum().sum()
print(f"   - 매장이 전체 구매의 {store_ratio:.1%}를 차지 → 매장 내 크로스셀링 유지")

print("\n3. 기대 효과:")
top_rate = best_segment.head(1)['반응률'].values[0]
overall_rate = df['Campaign_Response'].mean()
print(f"   - 현재 전체 캠페인 반응률: {overall_rate:.1%}")
print(f"   - 최고 그룹 반응률: {top_rate:.1%} (전체 대비 {top_rate/overall_rate:.1f}배)")
print(f"   - 2% 개선을 확인하기 위한 A/B 테스트에는 그룹당 {int(np.ceil(n_design)):,}명이 필요합니다")

print("\n4. 한계점:")
print("   - 관찰 데이터 기반으로 인과관계 주장 불가 (특히 A/B 테스트 결과)")
print("   - 데이터 수집 시점이 오래되었을 수 있어, 현재 고객 행동과 다를 수 있음")
print("   - 외부 변수(경기, 계절, 경쟁사 활동 등) 미통제")
print("   - 생존자 편향: 이탈 고객 데이터가 없어 잔존 고객만 분석됨")
print("   - 교육 수준의 효과(η²=0.014)는 실무적으로 미미함 → 고객 분류 변수로 부적합")